Trying to implement WGAN as it is : https://github.com/martinarjovsky/WassersteinGAN/blob/master/main.py

In [1]:
from __future__ import print_function
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.utils import data
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os
import json
import h5py
from pathlib import Path
import numpy as np
import models.HDF5Dataset as H
from torchsummary import summary

In [ ]:
## Load and make them iterable
loader_params = {'batch_size': 10, 'shuffle': True, 'num_workers': 6}
d = H.HDF5Dataset('/home/eren/WassersteinGAN/gamma-test.hdf5', 'layers')
dataloader = data.DataLoader(d, **loader_params)
data_iter = iter(dataloader)


In [ ]:
data = data_iter.next()

In [ ]:
data.shape

### Testing discriminator

In [3]:
import models.dcgan as dcgan
ndf = 64
nc = 5
netD = dcgan.DCGAN_D(16, nc, ndf, 1, 5)
print(netD)

DCGAN_D(
  (main): Sequential(
    (initial:5-64:conv): Conv2d(5, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (initial:64:relu): LeakyReLU(negative_slope=0.2, inplace)
    (extra-layers-0:64:conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (extra-layers-0:64:batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (extra-layers-0:64:relu): LeakyReLU(negative_slope=0.2, inplace)
    (extra-layers-1:64:conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (extra-layers-1:64:batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (extra-layers-1:64:relu): LeakyReLU(negative_slope=0.2, inplace)
    (extra-layers-2:64:conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (extra-layers-2:64:batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

### Testing Generator 

In [8]:
ngf = 64
nz = 1024
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
netG = dcgan.DCGAN_G(16, nc, nz, ngf,1,5 ).to(device)
print(netG)
#for i in range(len(list(netG.parameters()))):
#    print (list(netG.parameters())[i].size())

#summary(netG, input_size=(5, 16, 16))

netG_total_params = sum(p.numel() for p in netG.parameters() if p.requires_grad)
netG_total_params

DCGAN_G(
  (main): Sequential(
    (initial:1024-128:convt): ConvTranspose2d(1024, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (initial:128:batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (initial:128:relu): ReLU(inplace)
    (pyramid:128-64:convt): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid:64:batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid:64:relu): ReLU(inplace)
    (extra-layers-0:64:conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (extra-layers-0:64:batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (extra-layers-0:64:relu): ReLU(inplace)
    (extra-layers-1:64:conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (extra-layers-1:64:batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin

2418688

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        


In [ ]:
netD.apply(weights_init)

In [ ]:
netG.apply(weights_init)

#### one forward pass 

In [ ]:
input = torch.FloatTensor(1000, nc, 16, 16)
noise = torch.FloatTensor(1000, nz, 1, 1)
noise.resize_(1000, nz, 1, 1).normal_(0, 1)
input.resize_as_(data.float()).copy_(data.float())
inputv = Variable(input)
noisev = Variable(noise)

In [ ]:
netD(inputv)

In [ ]:
netG(noisev)[0].shape

In [ ]:
fake = Variable(netG(noisev).data)

In [ ]:
fake.shape